In [ ]:
import numpy as np
from scipy.spatial import distance
from matplotlib import pyplot as plt
import time
import copy
#from carpeta.nombredelarchico import nombreClase
import sys
sys.path.insert(0, "/home/ktita/Documents/Projeto-Experimentos/medidas_internas")
from Intra_cluster_average_similarity import Intra_cluster_similarity
from Inter_cluster_centroids_average_similarity import Inter_cluster_similarity
from Davies_bouldin import Davies_bouldin
from Medidas import Medidas
from biclustering.biclustering import DeltaBiclustering
from biclustering.biclustering import MSR

import sys
sys.path.insert(0, "/home/ktita/Documents/Projeto-Experimentos/dataSetSintetic/")
from SyntheticData import SyntheticData

In [ ]:
#Cuidado con los valores quando K e L = a
params = {
    'k' : {
        'a' : 1, 'b' : 3, 'd' : 3, 'e': 3, 'f': 4, 'g': 4, 'h': 5, 'i': 4
    },
    'l' : {
        'a' : 1, 'b' : 3, 'd' : 3, 'e': 3, 'f': 4, 'g': 4, 'h': 5, 'i': 4
    }
}

In [ ]:
#Dada I e J (lineas e colunas) del grupo o cogrupo obtiene el grupo (g) o cogrupo (c) de X dependiendo de la opcion
def get_grupo_cogrupo(X, I, J, option):
    I.sort()
    J.sort()
    if option == 'g':
        grupo = X[I]
        return grupo
    elif option == 'c':
        cogrupo = X[I][:,J]
        return cogrupo

In [ ]:
#Obter Is e Js quando Matriz es generada sin suffle
def obterIsJs(x_row, y_col):
    Is = []
    Js = []
    for p in range(len(x_row)):
        i = range(int (x_row[p][0]),int (x_row[p][1]))
        j = range(int (y_col[p][0]),int (y_col[p][1]))        
        Is.append(i)
        Js.append(j)
    return (Is, Js)

In [ ]:
#Obtiene los deltas adecuados para cada iteracion segun su conjunto de dados ideal, crea una lista de deltas.
def obter_lst_delta(X, Is, Js):
    num_cogrupos = len(Is)
    lst_msr = []
    for p in range (num_cogrupos):
        cogrupo = get_grupo_cogrupo(X, Is[p], Js[p], 'c')
        lst_msr.append(MSR(cogrupo).H)
    
    min_msr = min(lst_msr)
    lst_delta = [min_msr/10 ,min_msr, 10*min_msr]
    return lst_delta

In [ ]:
#Funcion retorna un array de indeces de i(cluster) e los numeros de i
##Usada para guardar no formato: Is = ([[2, 0, 1], [4, 3, 2], [5, 6, 4], [1]])
#lst_index_is= [0, 0, 0, 1, 1, 1, 2, 2, 2, 3] (equivale a los cluster)
#lst_is [2, 0, 1, 4, 3, 2, 5, 6, 4, 1]
def  obter_formato_is_js(As):
    lst_index_as = []
    lst_as = []
    for i in range(len(As)):
        for j in range(len(As[i])):
            lst_index_as.append(i)
            lst_as.append(As[i][j])
    return (lst_index_as, lst_as)

In [ ]:
lst_sparsity = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 98]
#lst_sparsity = [70]
#lst_alpha = [1, 1.2]
lst_alpha = [1, 1.2, 1.5, 2, 2.5, 3]

lst_inter_cluster_g = []
lst_inter_cluster_c = []
lst_intra_cluster_g = []
lst_intra_cluster_c = []
lst_indice_bouildin_g = []
lst_indice_bouildin_c = []

lst_average_volume = []
lst_average_MSR = []
lst_percentage_coverage = []
lst_percentage_occupancy = []
lst_percentage_overlap = []

lst_tempo = []
lst_nome = []

for data_format_option in ['a', 'b', 'd', 'e', 'f', 'g', 'h', 'i']:
    data_orig,  x_row, y_col = SyntheticData().generate_dataset(data_format_option, noise=1, noise_background=False, shuffle=False,  esparsidad=False, niv_spar = 0)
    rows_orig, columns_orig = data_orig.shape 
    #X = data_orig[0:rows_orig-1][:,0:columns_orig-1]
    #n, m = X.shape
    
    #Obter a lista de delta
    Is_inicio, Js_inicio = obterIsJs(x_row, y_col)
    lst_delta = obter_lst_delta(data_orig, Is_inicio, Js_inicio)

    
    for a in lst_sparsity:
        #a -> usado para la esparsidad
        data_sparse = SyntheticData().generate_sparsity(data_orig,a)
        # ---------------------------- Guardando Graficas e Matrices --------------------------------
        #Salvar las graficas
        plt.matshow(data_sparse, cmap=plt.cm.Purples)
        plt.tick_params(labeltop=False, labelbottom=False, labelright=False, labelleft=False)
        plt.savefig('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/Matrix/%s-structure-%s-sparsity.png' % (data_format_option,a))
        plt.title('%s-structure-%s-sparsity.png' % (data_format_option,a))
        plt.show()
        
        #Salvar las matrices
        nome = '' + str(data_format_option) +'-structure-' + str(a) + '-sparsity.csv'
        np.savetxt('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/Matrix/' + nome , data_sparse, fmt="%.5f", delimiter=',')

        #Obter a lista de delta
        #Is_inicio, Js_inicio = obterIsJs(x_row, y_col)
        #lst_delta = obter_lst_delta(data_sparse, Is_inicio, Js_inicio)
        
        data_suffle = SyntheticData().generate_suffle(data_sparse)
        #Salvar las matrices con suffle
        nome = '' + str(data_format_option) +'-structure-' + str(a) + '-sparsity-suffle.csv'
        np.savetxt('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/Matrix/' + nome , data_suffle, fmt="%.5f", delimiter=',')
        
        plt.matshow(data_suffle, cmap=plt.cm.Purples)
        plt.tick_params(labeltop=False, labelbottom=False, labelright=False, labelleft=False)
        plt.savefig('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/Matrix/%s-structure-%s-sparsity-suffle.png' % (data_format_option,a))
        plt.title('%s-structure-%s-sparsity-suffle.png' % (data_format_option,a))
        plt.show()
                
        #Matriz con suffle sin las etiquetas de lineas e colunas
        X = data_suffle[0:rows_orig-1][:,0:columns_orig-1]
        n, m = X.shape
        nome = '' + str(data_format_option) +'-structure-' + str(a) + '-sparsity-X.csv'

        np.savetxt('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/Matrix/' + nome , X, fmt="%.5f", delimiter=',')

        # ------------------------------------------------------------------------------------------
        num_cogrupos = params['k'][data_format_option]
        for b in lst_alpha:
            #b -> usado para alpha
            alpha = b
            
            for c in lst_delta:
                biclusters = []
                Is = [] 
                Js = []
                #c -> usado para delta 
                delta = c
                # ------------------------------- Llamada al algoritmo --------------------------------------
                inicio = time.time()
                biclusters , Is, Js = DeltaBiclustering().find_biclusters(X, delta, alpha, num_cogrupos)

                fim = time.time()
                lst_tempo.append(fim - inicio)
                print("Tempo: %s seconds " % (fim - inicio))

                nome = 'structure_' + str(data_format_option) + '_num-cogrupos_' + str(num_cogrupos) + '_niv-sparsity_' + str(a) + '_alpha_' + str(alpha) +  '_delta_' + str(delta)
                lst_nome.append(nome)

                # Guardando los Is e Js retornados del algoritmo!
                lst_index_Is, lst_Is = obter_formato_is_js(Is)
                lst_index_Js, lst_Js = obter_formato_is_js(Js)
                archivo_is = np.array(zip(lst_index_Is, lst_Is), dtype=[('var1', 'int'), ('var2', 'int')])
                np.savetxt('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/Is-Js/Is-'+ nome + '.csv', archivo_is, fmt='%d', header='Cluster,Linhas_Is', delimiter=',')
                archivo_js = np.array(zip(lst_index_Js, lst_Js), dtype=[('var1', 'int'), ('var2', 'int')])
                np.savetxt('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/Is-Js/Js-' + nome + '.csv', archivo_js, fmt='%d', header='Cluster,Linhas_Js', delimiter=',')
                
                #Guardando los biclusters
                for bic in range(len(biclusters)):
                    np.savetxt('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/bicluster/'+ nome + '- bic-' + str(bic) + '.csv', biclusters[bic], fmt='%.5f', delimiter=',')
                    
                
                # ------------------------------- Medicion del los indices -------------------------------
                lst_inter_cluster_g.append(Inter_cluster_similarity().indice_inter_cluster(X, Is, Js, 'g'))
                lst_inter_cluster_c.append(Inter_cluster_similarity().indice_inter_cluster(X, Is, Js, 'c'))

                lst_intra_cluster_g.append(Intra_cluster_similarity().indice_intra_cluster(X, Is, Js, 'g'))
                lst_intra_cluster_c.append(Intra_cluster_similarity().indice_intra_cluster(X, Is, Js, 'c'))

                davies_bouldin_grupo = Davies_bouldin().indice_davies_bouldin(X, Is, Js, 'g')
                davies_bouldin_cogrupo =  Davies_bouldin().indice_davies_bouldin(X, Is, Js, 'c')
                lst_indice_bouildin_g.append(davies_bouldin_grupo)
                lst_indice_bouildin_c.append(davies_bouldin_cogrupo)

                lst_average_volume.append(Medidas().get_average_volume(Is, Js))
                lst_average_MSR.append(Medidas().get_average_MSR(X, Is, Js))
                lst_percentage_coverage.append(Medidas().get_percentage_coverage(X, Is, Js))
                lst_percentage_occupancy.append(Medidas().get_percentage_occupancy(X, Is, Js))
                lst_percentage_overlap.append(Medidas().get_percentage_overlap(X, Is, Js))


    
#---- Guarda el archivo de todas las medidas
arquivo_medidas = np.array(zip(lst_nome, lst_inter_cluster_g, lst_inter_cluster_c, lst_intra_cluster_g, lst_intra_cluster_c, lst_indice_bouildin_g, lst_indice_bouildin_c, lst_average_volume, lst_average_MSR, lst_percentage_coverage, lst_percentage_occupancy, lst_percentage_overlap, lst_tempo), dtype=[('var1', 'object'), ('var2', 'float'), ('var3', 'float'), ('var4', 'float'), ('var5', 'float'), ('var6', 'float'), ('var7', 'float'), ('var8', 'float'), ('var9', 'float'), ('var10', 'float'), ('var11', 'float'), ('var12', 'float'), ('var13', 'float')])
np.savetxt('/home/ktita/Documents/Projeto-Experimentos/Testes-Cheng/test1_cheng.csv', arquivo_medidas, fmt="%s, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f", header='Nome,Inter_cluster_g,Inter_cluster_c,Intra_cluster_g,Intra_cluster_c,Indice_bouldin_g,Indice_buildin_c,Average_volume,Average_msr,Percentage_coverage,Percentage_occupancy,Percentage_overlap,Tempo_seg', delimiter=',')
    